## Import Dependencies

In [1]:
%%time

import sys
import gpytorch
import numpy as np
import pandas as pd
import torch
from datetime import datetime
from scipy.stats import qmc
import itertools
from itertools import combinations_with_replacement, combinations, permutations

import bo_methods_lib
from bo_methods_lib.bo_functions_generic import gen_theta_set, clean_1D_arrays
from bo_methods_lib.GPBO_Classes_New import * #Fix this later
from bo_methods_lib.GPBO_Class_fxns import * #Fix this later

/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/Toy_Problem_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CPU times: user 2.36 s, sys: 297 ms, total: 2.65 s
Wall time: 24.3 s


## Define Method and DateTime

In [2]:
#Generate Method (except last part)
meth_name = Method_name_enum(1)
method = GPBO_Methods(meth_name)

print("Method Name: ", method.method_name.name)
print("Emulator?: ", method.emulator)
print("Obj Func: ", method.obj.name)
print("Sparse Grid?: ", method.sparse_grid)

#Set Date and Time
dateTimeObj = datetime.now()
timestampStr = dateTimeObj.strftime("%d-%b-%Y (%H:%M:%S)")
# print("Date and Time: ", timestampStr)
# DateTime = dateTimeObj.strftime("%Y/%m/%d/%H-%M-%S%p")
DateTime = dateTimeObj.strftime("%Y/%m/%d/%H-%M")
DateTime = None ##For Testing

Method Name:  C2
Emulator?:  True
Obj Func:  OBJ
Sparse Grid?:  True


## Define Case Study Simulator & Functions

In [3]:
def simulator_helper_test_fxns(cs_name, indecies_to_consider):
    """
    Sets the model for calculating y based off of the case study identifier.

    Parameters
    ----------
    cs_name: Class, The name/enumerator associated with the case study being evaluated

    Returns
    -------
    calc_y_fxn: function, the function used for calculation is case study cs_name.name
    """
    #Note: Add your function name from GPBO_Class_fxns.py here
    if cs_name.value == 1:
        dim_x = 1       
        theta_ref = np.array([1, -1])
        theta_names = ['theta_1', 'theta_2']
        calc_y_fxn = calc_cs1_polynomial
        
    elif cs_name.value == 2:
        dim_x = 2       
        theta_ref = np.array([-200,-100,-170,15,-1,-1,-6.5,0.7,0,0,11,0.6,-10,-10,-6.5,0.7,1,0,-0.5,-1,0,0.5,1.5,1])
        theta_names = ['A_1', 'A_2', 'A_3', 'A_4', 'a_1', 'a_2', 'a_3', 'a_4', 'b_1', 'b_2', 'b_3', 'b_4', 'c_1', 
                       'c_2', 'c_3', 'c_4', 'x0_1', 'x0_2', 'x0_3', 'x0_4', 'x1_1', 'x1_2', 'x1_3', 'x1_4']
        calc_y_fxn = calc_muller
        
    else:
        raise ValueError("self.CaseStudyParameters.cs_name.value must exist!")

    return Simulator(dim_x, 
                     indecies_to_consider, 
                     theta_ref,
                     theta_names,
                     calc_y_fxn)

## Define Case Study

In [4]:
cs_name  = CS_name_enum(2)
print("CS Name: ", cs_name.name)

indecies_to_consider = list(range(4, 12)) #This is what changes for different subproblems of CS2

ep0 = 1
sep_fact = 1.0
normalize = False
num_x_data = 27
num_theta_data = 20
lhs_gen_theta = True
eval_all_pairs = False
noise_mean = 0
noise_std = 0.01
kernel = Kernel_enum(1)
set_lenscl = 1
outputscl = False
retrain_GP = 2
GP_train_iter = 300
bo_iter_tot = 3
bo_run_tot = 2
save_fig = False
save_data = False
num_data = None
seed = 1

CS = CaseStudyParameters(cs_name, ep0, sep_fact, normalize, eval_all_pairs, noise_mean, noise_std, bo_iter_tot, 
                         bo_run_tot, save_fig, save_data, DateTime, seed)

simulator = simulator_helper_test_fxns(cs_name, indecies_to_consider)

CS Name:  CS2


## Generate Experimental X and Y Data (If not given)

In [5]:
bounds_x_l = [-1.5, -0.5]
bounds_x_u = [1, 2]
bounds_p_l = [-2, -2, -10, -2, -2, -2,  5, -2]
bounds_p_u = [ 2,  2,   0,  2,  2,  2, 15,  2]
gen_meth_x = Gen_meth_enum(1)

# exp_data = Data(true_params, x_data, None, None, None, None, None, None)
driver = GPBO_Driver(CS, simulator, None, None)
exp_data = driver.gen_exp_data(bounds_p_l, bounds_x_l, bounds_p_u, bounds_x_u, num_x_data, gen_meth_x)

# print(exp_data.y_vals)

## Normalize Experimental X Data (optional)

In [6]:
scaled_exp_data = exp_data
bounds_p = scaled_exp_data.bounds_theta
bounds_x = scaled_exp_data.bounds_x

scaled_exp_data.theta_vals = scaled_exp_data.normalize(simulator.theta_true, bounds_p)
scaled_exp_data.x_vals = scaled_exp_data.normalize(scaled_exp_data.x_vals, bounds_x)

# print(scaled_exp_data.theta_true)
# print(scaled_exp_data.x_vals)

## Generate Simulation Theta and y Data

In [7]:
#Generate and store data as a method in the driver class which creates an instances of the data class and adds data to it)
share_x_from_exp = True
gen_meth_x = Gen_meth_enum(1)
gen_meth_theta = Gen_meth_enum(1)
sim_data = driver.gen_sim_data(method, num_theta_data, gen_meth_theta, num_x_data, gen_meth_x, share_x_from_exp)

#Set number and dimension of training data
num_gp_data = sim_data.get_num_gp_data(method)
dim_gp_data = sim_data.get_dim_gp_data(method)
num_theta_calc = sim_data.get_num_theta()

print(sim_data.get_num_theta(), sim_data.get_num_x_vals(), num_gp_data)
# print(sim_data.y_vals)
# print(num_gp_data, dim_gp_data, num_theta_calc)

540 540 540


## Generate Type 1 Training Data

## Shuffle and split training data

## Build GP Model

In [8]:
gp_emulator = GP_Emulator(CS, kernel, set_lenscl, outputscl, retrain_GP, GP_train_iter)

## Augmenting Training Data (Skipped Steps)

In [9]:
best_theta = Data(None, exp_data.x_vals, bounds_p_l, bounds_x_l, bounds_p_u, bounds_x_u, None, None, None, None, None)

#Note: Will generate this using a method in driver that doesn't exist yet but generates experimental data instead of sim data
if "2" in method.method_name.name:
    best_theta.theta_vals = np.repeat(simulator.theta_true.reshape(1,-1), exp_data.get_num_x_vals(), axis = 0)
    best_theta.y_vals = simulator.create_sim_data(CS, best_theta)
else:
    best_theta.theta_vals = simulator.theta_true.reshape(1,-1)
    best_theta.y_vals = simulator.create_sse_sim_data(method, CS, best_theta, exp_data)

print(len(sim_data.theta_vals), len(sim_data.y_vals))
sim_data = driver.augment_train_data(sim_data, best_theta) #Best error in driver class
print(len(sim_data.theta_vals), len(sim_data.y_vals))

540 540
567 567


# Case Study 1 Test

In [10]:
cs_name  = CS_name_enum(1)

#Want to make this all one array. I think that makes more sense
indecies_to_consider = list(range(0, 2)) #This is what changes for different subproblems of CS2
ep0 = 1
sep_fact = 1.0
normalize = False
num_x_data = 5
dim_x = 1
num_theta_data = 20
lhs_gen_theta = True
eval_all_pairs = False
noise_mean = 0
noise_std = 0.01
kernel = Kernel_enum(1)
set_lenscl = 1
outputscl = 1
retrain_GP = 2
GP_train_iter = 300
bo_iter_tot = 3
bo_run_tot = 2
save_fig = False
save_data = False
x_data_vals = np.linspace(-2,2,num_x_data)
num_data = None
seed = 1

CS = CaseStudyParameters(cs_name, ep0, sep_fact, normalize, eval_all_pairs, noise_mean, noise_std, bo_iter_tot, 
                         bo_run_tot, save_fig, save_data, DateTime, seed)

simulator = simulator_helper_test_fxns(cs_name, indecies_to_consider)
gp_emulator = GP_Emulator(CS, kernel, set_lenscl, outputscl, retrain_GP, GP_train_iter)

bounds_x_l = [-2]
bounds_x_u = [2]
bounds_p_l = [-2, -2]
bounds_p_u = [ 2,  2]
gen_meth_x = Gen_meth_enum(2)

# exp_data = Data(true_params, x_data, None, None, None, None, None, None)
driver = GPBO_Driver(CS, simulator, None, None)
exp_data = driver.gen_exp_data(bounds_p_l, bounds_x_l, bounds_p_u, bounds_x_u, num_x_data, gen_meth_x)

# print(exp_data.x_vals)

#Generate and store data as a method in the driver class which creates an instances of the data class and adds data to it)
share_x_from_exp = True
gen_meth_x = Gen_meth_enum(2)
gen_meth_theta = Gen_meth_enum(2)
sim_data = driver.gen_sim_data(method, num_theta_data, gen_meth_theta, num_x_data, gen_meth_x, share_x_from_exp)

#Set number and dimension of training data
num_gp_data = sim_data.get_num_gp_data(method)
dim_gp_data = sim_data.get_dim_gp_data(method)
num_theta_calc = sim_data.get_num_theta()

print(sim_data.get_num_theta(), sim_data.get_num_x_vals(), num_gp_data)

gp_emulator = GP_Emulator(CS, kernel, set_lenscl, outputscl, retrain_GP, GP_train_iter)

best_theta = Data(None, exp_data.x_vals, bounds_p_l, bounds_x_l, bounds_p_u, bounds_x_u, None, None, None, None, None)

# #Note: Will generate this using a method in driver that doesn't exist yet but generates experimental data instead of sim data
if "2" in method.method_name.name:
    best_theta.theta_vals = np.repeat(simulator.theta_true.reshape(1,-1), exp_data.get_num_x_vals(), axis = 0)
    best_theta.y_vals = simulator.create_sim_data(CS, best_theta)
else:
    best_theta.theta_vals = simulator.theta_true.reshape(1,-1)
    best_theta.y_vals = simulator.create_sse_sim_data(method, CS, best_theta, exp_data)

print(len(sim_data.theta_vals), len(sim_data.y_vals))
sim_data = driver.augment_train_data(sim_data, best_theta) #Best error in driver class
print(len(sim_data.theta_vals), len(sim_data.y_vals))

2000 2000 2000
2000 2000
2005 2005
